# Summary:

Demo of:
* Getting the n most frequent items in a group (eg: a groupby of another column)
* Returning those n most freq as rows in a dataframe
* Unstacking those n most freq to be columns of a dataframe
* Getting all unique items in each group, returned as columns of a dataframe ordered by frequency

In [1]:
import pandas as pd
import numpy as np

In [2]:
np.random.seed(44)
n = 12
data = {
    'Description': [f'D{i}' for i in np.random.randint(0, 3, n)],
    'Category': [f'C{i}' for i in np.random.randint(0, 3, n)],
}

In [3]:
df = pd.DataFrame(data)

In [4]:
df.sort_values(["Description", "Category"])

,Description,Category
0,D0,C0
6,D0,C0
3,D0,C1
4,D0,C2
11,D1,C0
1,D1,C1
2,D1,C1
8,D1,C1
10,D1,C2
5,D2,C0


# Frequency counts for each category in groups of descriptions, sorted

In [5]:
# Group by Category, select description, and get the value_counts of that series
# These counts are implicitly sorted
cat_by_freq = df.groupby("Description")["Category"].value_counts()

In [6]:
cat_by_freq

Description  Category
D0           C0          2
             C1          1
             C2          1
D1           C1          3
             C0          1
             C2          1
D2           C0          2
             C1          1
Name: Category, dtype: int64

**NOTE:** In each of the above Series, both the index with our original categories (C0, C1, C2) and our new value_count column are named *Category*.  So let's rename it to avoid annoyance later.  We can do that a few different ways

In [7]:
%%timeit
# Like before.  Just for interest
cat_by_freq = df.groupby("Description")["Category"].value_counts()

1.11 ms ± 15.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [8]:
%%timeit
# Added .to_frame at the end just because the next step implicitly converts to frame 
cat_by_freq = df.groupby("Description")["Category"].value_counts().rename("category_counts", inplace=True).to_frame()

1.61 ms ± 177 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [9]:
%%timeit
cat_by_freq = df.groupby("Description")["Category"].agg(category_counts='value_counts')

2.82 ms ± 214 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


So probably makes sense to do .rename() for our purposes here.  

In [10]:
cat_by_freq = df.groupby("Description")["Category"].value_counts().rename("category_counts", inplace=True)

In [11]:
# cat_by_freq is a series with multi-index of (Category, Description)
print(type(cat_by_freq))
print(cat_by_freq.index)

<class 'pandas.core.series.Series'>
MultiIndex([('D0', 'C0'),
            ('D0', 'C1'),
            ('D0', 'C2'),
            ('D1', 'C1'),
            ('D1', 'C0'),
            ('D1', 'C2'),
            ('D2', 'C0'),
            ('D2', 'C1')],
           names=['Description', 'Category'])


In [12]:
# could also group by pairs of c-d, then get counts via size, but it wont be ordered
# counts = df.groupby("Description")["Category"].size()

### As a series

In [13]:
cat_by_freq

Description  Category
D0           C0          2
             C1          1
             C2          1
D1           C1          3
             C0          1
             C2          1
D2           C0          2
             C1          1
Name: category_counts, dtype: int64

### As a dataframe

In [14]:
cat_by_freq.to_frame()

category_counts
Description Category                 
D0          C0                      2
            C1                      1
            C2                      1
D1          C1                      3
            C0                      1
            C2                      1
D2          C0                      2
            C1                      1

In [15]:
# Without index
cat_by_freq.reset_index()

,Description,Category,category_counts
0,D0,C0,2
1,D0,C1,1
2,D0,C2,1
3,D1,C1,3
4,D1,C0,1
5,D1,C2,1
6,D2,C0,2
7,D2,C1,1


If we wanted a 2D table showing the counts for each pair, we could always unstack counts.

Unstack by default takes the -1'th index and turns it into column headings.  It can also take a different level as an arg

In [16]:
cat_by_freq.unstack(level=-1)

Category,C0,C1,C2
Description,,,
D0,2.0,1.0,1.0
D1,1.0,3.0,1.0
D2,2.0,1.0,NaN


### Top n in each group

In [17]:
cat_by_freq.groupby("Description").head(1)

Description  Category
D0           C0          2
D1           C1          3
D2           C0          2
Name: category_counts, dtype: int64

# Dataframe with rows of Description and Columns of \[Most_freq_cat, second_most_freq_cat, ...\] ...

To do this, we:

* get grouped frequency counts like above, as a dataframe with the index reset
* grab the Category names (we don't need the counts)
* do a second groupby on Description
* For each group, we:
    * Turn each group (which start as Series of Category names, indexed by the row number of that record in the original cat_by_freq.reset_index()) into a new Series indexed by the integer position in each group, eg: 0 to n-cat_in_this_group.  When the groupby receives these series, it adds the Description index on to create a multiindex series with (Description, rank) (although the "rank" index has no name by default)
* unstack the series, moving the i index to column names
* rename the columns to make it more friendly :)

In [18]:
# Intermediate product just to make it clear
temp = cat_by_freq.reset_index().groupby("Description")["Category"].apply(lambda x: pd.Series(x.values))
print(temp)
print("type: ", type(temp))
print("index: ", temp.index)

Description   
D0           0    C0
             1    C1
             2    C2
D1           0    C1
             1    C0
             2    C2
D2           0    C0
             1    C1
Name: Category, dtype: object
type:  <class 'pandas.core.series.Series'>
index:  MultiIndex([('D0', 0),
            ('D0', 1),
            ('D0', 2),
            ('D1', 0),
            ('D1', 1),
            ('D1', 2),
            ('D2', 0),
            ('D2', 1)],
           names=['Description', None])


Altogether...

In [19]:
cats_ranked = cat_by_freq.reset_index().groupby("Description")["Category"].apply(lambda x: pd.Series(x.values)).unstack(-1)

In [20]:
cats_ranked

,0,1,2
Description,,,
D0,C0,C1,C2
D1,C1,C0,C2
D2,C0,C1,NaN


Note that we could have filled Nones in with values during the unstack, and we could also rename the columns with rename

# Dataframe with top n most frequent

Similar to above, but with head()

In [21]:
np.random.seed(44)
n_rows = 100
n_columns = 10
data = {
    'Description': [f'D{i}' for i in np.random.randint(0, n_columns, n_rows)],
    'Category': [f'C{i}' for i in np.random.randint(0, n_columns, n_rows)],
}

In [22]:
df = pd.DataFrame(data)

In [23]:
n_most_freq = 2
df_n_most_freq = (df
                  .groupby("Description")["Category"]
                  .value_counts()  
                  .rename("category_counts") # Series of cat_count vs (Description,Cat), within each group sorted by cat_count
                  .reset_index()             # As a dataframe with Desc and Cat as columns
                  .groupby("Description").head(n_most_freq)  # Keeping only the top n from each group
                  .groupby("Description")["Category"]
                  .apply(lambda x: pd.Series(x.values))  # Building the cat vs rank structure
                  .unstack(-1)  # And changing to a dataframe
)

In [24]:
df_n_most_freq

,0,1
Description,,
D0,C0,C1
D1,C2,C5
D2,C8,C1
D3,C6,C1
D4,C6,C8
D5,C1,C2
D6,C1,C5
D7,C0,C1
D8,C1,C2


In [25]:
n_most_freq = 10
df_n_most_freq = (df
                  .groupby("Description")["Category"]
                  .value_counts()  
                  .rename("category_counts") # Series of cat_count vs (Description,Cat), within each group sorted by cat_count
                  .reset_index()             # As a dataframe with Desc and Cat as columns
                  .groupby("Description").head(n_most_freq)  # Keeping only the top n from each group
                  .groupby("Description")["Category"]
                  .apply(lambda x: pd.Series(x.values))  # Building the cat vs rank structure
                  .unstack(-1)  # And changing to a dataframe
)

In [26]:
df_n_most_freq

,0,1,2,3,4,5,6,7,8
Description,,,,,,,,,
D0,C0,C1,C2,C3,C4,C8,NaN,NaN,NaN
D1,C2,C5,C0,C3,C6,NaN,NaN,NaN,NaN
D2,C8,C1,C5,C6,C0,C2,C7,C9,NaN
D3,C6,C1,C4,C7,C0,C8,C9,NaN,NaN
D4,C6,C8,C9,C0,C2,C3,C5,C7,NaN
D5,C1,C2,C4,C7,C8,C9,NaN,NaN,NaN
D6,C1,C5,C7,C0,C6,NaN,NaN,NaN,NaN
D7,C0,C1,C4,C5,C7,C9,NaN,NaN,NaN
D8,C1,C2,C5,C9,C0,C4,C6,C7,C8


As a function...

In [44]:
def get_grouped_frequency_series(df, by, col_to_count, n_most_frequent=None):
    """
    Returns the n most frequent occurrences of col_to_count in each group of by in df
    
    Ex: 
        df = pd.DataFrame({"x": [f"x{i}" for i in [0, 0, 0, 0, 1, 1, 1]],
                           "y": [f"y{i}" for i in [0, 1, 1, 2, 1, 2, 2]]
                           })
        get_grouped_frequency_series(df, "x", "y", 2)

    Results in:
            x   y   y_counts
        0   x0  y1  2
        1   x0  y0  1
        3   x1  y2  2
        4   x1  y1  1
        
    See get_most_frequent_as_df for description of args
    """
    result = (df
        .groupby(by)[col_to_count]
        .value_counts()  
        .rename(f"{col_to_count}_counts") # Series of cat_count vs (Description,Cat), within each group sorted by cat_count
        .reset_index()             # As a dataframe with Desc and Cat as columns
    )
    if n_most_frequent:
        result = result.groupby(by).head(n_most_frequent)  # Keeping only the top n from each group

    return result
        
def get_most_frequent_as_df(df, by, col_to_count, n_most_frequent=None, pad_to_n_columns=True):
    """
    Returns the n most frequent items in col_to_count for each group of by in df, as a dataframe where columns are in order of frequency rank
    
    Ex: 
        df = pd.DataFrame({"x": [f"x{i}" for i in [0, 0, 0, 0, 1, 1, 1]],
                           "y": [f"y{i}" for i in [0, 1, 1, 2, 1, 2, 2]]
                           })
        get_most_frequent_as_df(df, "x", "y", 2)
        
    Results in:
        
              0    1
        x
        x0   y1   y0
        x1   y2   y1
    
    Args:
        df (pd.DataFrame): Data to find frequent values from
        by (str): The name of the column to groupby when partitioning to find values
        col_to_count (str): The name of the column to count the value frequencies of
        n_most_frequent (int): Number of items to return per group
        pad_to_n_columns (bool): If True, will add columns if no group has 
                                 n_most_frequent unique entries. 
                                 If False, dataframe may return < n_most_frequent 
    
    Returns:
        A pd.DataFrame with rows of by groups and <=n_most_frequent columns of entries
        in col_to_count, from most frequent (leftmost column) to least frequent 
        (rightmost column)
    """
    # Get grouped as a series
    by_series = get_grouped_frequency_series(df, by, col_to_count, n_most_frequent)
    
    # Convert to the 2D frame
    df_returned = (by_series
        .groupby(by)[col_to_count]
        .apply(lambda x: pd.Series(x.values))  # Building the cat vs rank structure
        .unstack(-1)  # And changing to a dataframe
        )
    
    # Pad columns if needed
    if pad_to_n_columns and n_most_frequent:
        columns = range(n_most_frequent)
        df_returned = pad_df(df_returned, columns, inplace=True)
    return df_returned

def pad_df(df, columns, pad_with=np.nan, inplace=False):
    """
    Returns df padded by columns of pad_with for any column in columns that is not already a column in df
    
    Args:
        df (pd.DataFrame): DataFrame to pad
        columns (list): List of column names to ensure that the df returned has
        pad_with: Value to put into the columns added
        inplace (bool): If True, modify df in place.  Else, return a copy
    """
    df = df if inplace else df.copy()
    
    for c in columns:
        if c not in df:
            df[c] = pad_with
    return df

In [45]:
get_grouped_frequency_series(df, "Description", "Category", n_most_frequent=1)

,Description,Category,Category_counts
0,D0,C0,2
6,D1,C2,2
11,D2,C8,4
19,D3,C6,3
26,D4,C6,4
34,D5,C1,2
40,D6,C1,2
45,D7,C0,1
51,D8,C1,2
60,D9,C4,3


In [46]:
get_most_frequent_as_df(df, "Description", "Category", n_most_frequent=20, pad_to_n_columns=False)

,0,1,2,3,4,5,6,7,8
Description,,,,,,,,,
D0,C0,C1,C2,C3,C4,C8,NaN,NaN,NaN
D1,C2,C5,C0,C3,C6,NaN,NaN,NaN,NaN
D2,C8,C1,C5,C6,C0,C2,C7,C9,NaN
D3,C6,C1,C4,C7,C0,C8,C9,NaN,NaN
D4,C6,C8,C9,C0,C2,C3,C5,C7,NaN
D5,C1,C2,C4,C7,C8,C9,NaN,NaN,NaN
D6,C1,C5,C7,C0,C6,NaN,NaN,NaN,NaN
D7,C0,C1,C4,C5,C7,C9,NaN,NaN,NaN
D8,C1,C2,C5,C9,C0,C4,C6,C7,C8


In [47]:
get_most_frequent_as_df(df, "Description", "Category", n_most_frequent=20, pad_to_n_columns=True)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Description,,,,,,,,,,,,,,,,,,,,
D0,C0,C1,C2,C3,C4,C8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D1,C2,C5,C0,C3,C6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D2,C8,C1,C5,C6,C0,C2,C7,C9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D3,C6,C1,C4,C7,C0,C8,C9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D4,C6,C8,C9,C0,C2,C3,C5,C7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D5,C1,C2,C4,C7,C8,C9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D6,C1,C5,C7,C0,C6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D7,C0,C1,C4,C5,C7,C9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D8,C1,C2,C5,C9,C0,C4,C6,C7,C8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
df2 = pd.DataFrame({"x": [f"x{i}" for i in [0, 0, 0, 0, 1, 1, 1]],
                   "y": [f"y{i}" for i in [0, 1, 1, 2, 1, 2, 2]]
                   })

get_most_frequent_as_df(df2, "x", "y", 2)

,0,1
x,,
x0,y1,y0
x1,y2,y1


In [31]:
get_grouped_frequency_series(df2, "x", "y", 2)

,x,y,y_counts
0,x0,y1,2
1,x0,y0,1
3,x1,y2,2
4,x1,y1,1


In [36]:
x = df.values
x[:, 3] = np.nan
x

IndexError: index 3 is out of bounds for axis 1 with size 2